# Apache Spark / Toree

In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types.{DoubleType, StringType, StructField, StructType}

In [2]:
val spark = SparkSession.builder().config("spark.executor.memory", "2g")
                    .config("spark.driver.memory", "4g")
                    .master("spark://spark-master:7077").getOrCreate();

spark = org.apache.spark.sql.SparkSession@2cf65d04


org.apache.spark.sql.SparkSession@2cf65d04

In [3]:
//%AddJar file:/opt/spark-data/libs/xgb_1/xgboost4j_2.11-1.0.0.jar
//%AddJar file:/opt/spark-data/libs/xgb_1/xgboost4j-spark_2.11-1.0.0.jar

Name: Syntax Error.
Message: 
StackTrace: 

In [6]:
%AddDeps ml.dmlc xgboost4j_2.11 1.0.0
%AddDeps ml.dmlc xgboost4j-spark_2.11 1.0.0

Marking ml.dmlc:xgboost4j_2.11:1.0.0 for download
Obtained 2 files
Marking ml.dmlc:xgboost4j-spark_2.11:1.0.0 for download
Obtained 2 files


In [7]:
val schema = new StructType(Array(
  StructField("sepal_length", DoubleType, true),
  StructField("sepal_width", DoubleType, true),
  StructField("petal_length", DoubleType, true),
  StructField("petal_width", DoubleType, true),
  StructField("species", StringType, true)))

schema = StructType(StructField(sepal_length,DoubleType,true), StructField(sepal_width,DoubleType,true), StructField(petal_length,DoubleType,true), StructField(petal_width,DoubleType,true), StructField(species,StringType,true))


StructType(StructField(sepal_length,DoubleType,true), StructField(sepal_width,DoubleType,true), StructField(petal_length,DoubleType,true), StructField(petal_width,DoubleType,true), StructField(species,StringType,true))

In [8]:
val rawInput = spark.read
                    .option("header", "true")
                    .schema(schema).csv("/opt/spark-data/iris.csv")

rawInput = [sepal_length: double, sepal_width: double ... 3 more fields]


[sepal_length: double, sepal_width: double ... 3 more fields]

In [9]:
import org.apache.spark.ml.feature.StringIndexer
val stringIndexer = new StringIndexer().
  setInputCol("species").
  setOutputCol("classIndex").
  fit(rawInput)
val labelTransformed = stringIndexer.transform(rawInput).drop("species")

stringIndexer = strIdx_4751d88cbc34
labelTransformed = [sepal_length: double, sepal_width: double ... 3 more fields]


error: error while loading QualifiedTableName, class file '/home/jovyan/spark/jars/spark-catalyst_2.11-2.4.5.jar(org/apache/spark/sql/catalyst/QualifiedTableName.class)' has location not matching its contents: contains class QualifiedTableName
error: error while loading JavaTypeInference, class file '/home/jovyan/spark/jars/spark-catalyst_2.11-2.4.5.jar(org/apache/spark/sql/catalyst/JavaTypeInference.class)' has location not matching its contents: contains class JavaTypeInference
error: error while loading FunctionIdentifier, class file '/home/jovyan/spark/jars/spark-catalyst_2.11-2.4.5.jar(org/apache/spark/sql/catalyst/FunctionIdentifier.class)' has location not matching its contents: contains class FunctionIdentifier
error: error while loading DefinedByConstructorParams, class file '/home/jovyan/spark/jars/spark-catalyst_2.11-2.4.5.jar(org/apache/spark/sql/catalyst/DefinedByConstructorParams.class)' has location not matching its contents: contains class DefinedByConstructorParams
err

[sepal_length: double, sepal_width: double ... 3 more fields]

In [10]:
import org.apache.spark.ml.feature.VectorAssembler
val vectorAssembler = new VectorAssembler().
  setInputCols(Array("sepal_length", "sepal_width", "petal_length", "petal_width")).
  setOutputCol("features")
val xgbInput = vectorAssembler.transform(labelTransformed).select("features", "classIndex")

vectorAssembler = vecAssembler_c64c0979e891
xgbInput = [features: vector, classIndex: double]


error: error while loading PipelineStage, class file '/home/jovyan/spark/jars/spark-mllib_2.11-2.4.5.jar(org/apache/spark/ml/PipelineStage.class)' has location not matching its contents: contains class PipelineStage
error: error while loading PipelineModel, class file '/home/jovyan/spark/jars/spark-mllib_2.11-2.4.5.jar(org/apache/spark/ml/PipelineModel.class)' has location not matching its contents: contains class PipelineModel
error: error while loading Predictor, class file '/home/jovyan/spark/jars/spark-mllib_2.11-2.4.5.jar(org/apache/spark/ml/Predictor.class)' has location not matching its contents: contains class Predictor
error: error while loading PredictorParams, class file '/home/jovyan/spark/jars/spark-mllib_2.11-2.4.5.jar(org/apache/spark/ml/PredictorParams.class)' has location not matching its contents: contains class PredictorParams
error: error while loading Pipeline, class file '/home/jovyan/spark/jars/spark-mllib_2.11-2.4.5.jar(org/apache/spark/ml/Pipeline.class)' has l

[features: vector, classIndex: double]

In [11]:
import ml.dmlc.xgboost4j.scala.spark.XGBoostClassifier
import ml.dmlc.xgboost4j.scala.spark.TrackerConf

In [12]:
val xgbParam = Map("eta" -> 0.1f,
      "missing" -> -999,
      "objective" -> "multi:softprob",
      "num_class" -> 3,
      "num_round" -> 100,
      "num_workers" -> 2,
      "tracker_conf"-> TrackerConf(60 * 60 * 1000, "scala"))
val xgbClassifier = new XGBoostClassifier(xgbParam).
      setFeaturesCol("features").
      setLabelCol("classIndex")
xgbClassifier.setMaxDepth(2)

xgbParam = Map(num_workers -> 2, num_class -> 3, objective -> multi:softprob, num_round -> 100, missing -> -999, tracker_conf -> TrackerConf(3600000,scala), eta -> 0.1)
xgbClassifier = xgbc_1cc855528a0b


xgbc_1cc855528a0b

In [ ]:
val xgbClassificationModel = xgbClassifier.fit(xgbInput)

# Testing On Train

In [ ]:
xgbInput.select("features").collect()

In [ ]:
val trainPred = xgbClassificationModel.predict()